In [1]:
import openai
import pandas as pd
import tqdm
import glob

import warnings
warnings.filterwarnings("ignore")

In [2]:
with open("../apikey/apikey.txt", "r") as f:
    openai.api_key = f.readline().replace("\n", "")

In [3]:
def query_prompt(prompt, max_tokens=4000):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[
            {"role": "system", "content": "You are a professor at a science university and creating a exam for your students."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=0,
    )
    return response["choices"][0]["message"]["content"]

In [4]:
def get_df(fname):
    def f(categories):
        for cat in categories:
            for word in [
                "geology",
                "physics",
                "chemistry",
                "mathematical",
                "biology",
                "astronomy",
                "ecology",
                "genetics",
                "statistics",
                "theoretical"
            ]:
                if word.lower() in cat.lower():
                    return True
            
        return False
    
    def text_preprocess(text):
        return text.replace("===", "\n").replace("==", "\n")

    df = pd.read_parquet(fname)
    df_science = df[df["categories"].apply(f)]
    df_science["text"] = "title: " + df_science["title"] + "\n" + df_science["text"].apply(text_preprocess)
    return df_science.sample(len(df_science)//10)

In [5]:
files = glob.glob("../data/wikipedia/a.parquet")

In [6]:
import time

In [7]:
import pickle

In [8]:
import json

In [9]:
from datetime import datetime as dt
import os

In [10]:
texts = []

In [11]:
import traceback 
batch_size = 1

def make_prompt(series):
    prompt = f"""
You are an expert AI assistant who specializes in answering multiple-choice questions. You may use the context below if it helps you to answer the following multiple-choice question.
The output should be an array of json format, with "prompt" as the question statement, "A," "B," "C," "D," and "E" as choices, "answer" as the answer choice (one of A through E).

Context:
{series['text']}

"""
    return prompt

def f(series):
    if series["A"] != series["A"]:
        if type(series["choices"]) == dict:
            for key in ["A", "B", "C", "D", "E"]:
                series[key] = series["choices"][key]
        elif type(series["choices"] == list):
            for i, key in enumerate(["A", "B", "C", "D", "E"]):
                series[key] = series["choices"][i]
    return series

now_date = dt.now().strftime("%Y%m%d%H%M%S")

first = True
for file in files:
    if os.path.basename(file) in ["all.parquet"]:
        print(f"pass: {file}")
        continue
    df_science = get_df(file)
    
    for i in tqdm.tqdm(range(len(df_science)), desc=file):
        try:
            series = df_science.iloc[i]
            prompt = make_prompt(series)
            text = query_prompt(prompt)
            texts_json = json.loads(text)
            if first:
                print(texts_json)
                first = False
            if type(texts_json) == dict:
                text_json["wiki_id"] = series["id"]
                text_json["original_text"] = series["text"]
                texts.append(text_json)
            else:
                for text_json in texts_json:
                    text_json["wiki_id"] = series["id"]
                    text_json["original_text"] = series["text"]
                    texts.append(text_json)
        except Exception as e:
            print(e)
            traceback.print_exc()
            print(text)
        if i % 20 == 0:
            df_texts = pd.DataFrame(texts)
            df_texts = df_texts.apply(f, axis=1)

            df_texts.to_csv(f"output_gpt3.5_generate/{now_date}.csv")

../data/wikipedia/a.parquet:   0%|▍                                                                                                                     | 1/252 [00:03<14:22,  3.44s/it]

[{'prompt': 'Who is the theoretical physicist that expounded the thought experiment of Astrochicken?', 'A': 'John von Neumann', 'B': 'Rodney Brooks', 'C': 'Michio Kaku', 'D': 'Freeman Dyson', 'E': 'None of the above', 'answer': 'D'}]


Traceback (most recent call last):
  File "/tmp/ipykernel_54387/324541624.py", line 39, in <module>
    texts_json = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
../data/wikipedia/a.parquet:   1%|▉                                                                                                                     | 2/252 [00:06<13:24,  3.22s/it]

Expecting value: line 1 column 1 (char 0)
Which player holds the record for the most appearances for the Albania national football team? 

A) Erjon Bogdani
B) Lorik Cana
C) Etrit Berisha
D) Ansi Agolli
E) Altin Lala


../data/wikipedia/a.parquet:   1%|█▍                                                                                                                    | 3/252 [00:10<15:06,  3.64s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_54387/324541624.py", line 39, in <module>
    texts_json = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
../data/wikipedia/a.parquet:   2%|█▊                                                                                                                    | 4/252 [00:13<13:12,  3.19s/it]

...

../data/wikipedia/a.parquet:  78%|██████████████████████████████████████████████████████████████████████████████████████████▋                         | 197/252 [12:40<03:10,  3.47s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_54387/324541624.py", line 39, in <module>
    texts_json = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
../data/wikipedia/a.parquet:  79%|███████████████████████████████████████████████████████████████████████████████████████████▏                        | 198/252 [12:43<03:03,  3.40s/it]

Expecting value: line 1 column 1 (char 0)
Which of the following statements is true about the AIDA diabetes simulator?

A) It can be used for individual patient simulation and glycemic prediction.
B) It is only intended for insulin therapy planning.
C) It is primarily used for research purposes.
D) It is not suitable for teaching or self-learning.
E) It is available for download on the AIDA website.


../data/wikipedia/a.parquet:  79%|███████████████████████████████████████████████████████████████████████████████████████████▌                        | 199/252 [12:47<03:04,  3.49s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_54387/324541624.py", line 39, in <module>
    texts_json = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
../data/wikipedia/a.parquet:  79%|████████████████████████████████████████████████████████████████████████████████████████████                        | 200/252 [12:49<02:48,  3.24s/it]

Expecting value: line 1 column 1 (char 0)
Which of the following is NOT a type of assay based on the nature of the assay process?

A) End point assay
B) Kinetic assay
C) High throughput assay
D) Multiplex assay
E) Ligand binding assay

Answer: E) Ligand binding assay


../data/wikipedia/a.parquet: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 252/252 [15:40<00:00,  3.73s/it]


In [12]:
 df_texts.to_csv(f"output_gpt3.5_generate/{now_date}.csv")

In [13]:

df_texts["answer"].value_counts()

A                                    113
B                                     44
D                                     34
C                                     31
E                                     20
Aurora Max                             2
寶瓶座 (bǎo píng zuò)                     1
Voronezh University                    1
Stanley Autler and Charles Townes      1
0.4-1.5 million years                  1
Name: answer, dtype: int64